In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 64 
batch_size = 128
max_iters = 1000
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.2

In [2]:
chars = ""
with open('wizard_of_oz.txt','r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)

In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n=int(0.8*len(data))
train_data = data[:n]
valid_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+block_size+1]for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs')
print(x)
print('targets')
print(y)

inputs
tensor([[57, 65, 58,  ..., 66, 58, 72],
        [73, 61, 58,  ...,  1, 62, 67],
        [ 1, 44, 61,  ..., 71, 58, 57],
        ...,
        [56, 61,  1,  ...,  0, 32, 58],
        [70, 74, 62,  ...,  1, 57, 68],
        [74, 60, 61,  ...,  0, 61, 54]], device='cuda:0')
targets
tensor([[65, 58,  1,  ..., 58, 72,  1],
        [61, 58,  1,  ..., 62, 67,  1],
        [44, 61, 58,  ..., 58, 57,  1],
        ...,
        [61,  1, 66,  ..., 32, 58,  1],
        [74, 62, 71,  ..., 57, 68, 76],
        [60, 61, 73,  ..., 61, 54, 69]], device='cuda:0')


In [5]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is ',context,' target is ',target)

when input is  tensor([80])  target is  tensor(1)
when input is  tensor([80,  1])  target is  tensor(1)
when input is  tensor([80,  1,  1])  target is  tensor(28)
when input is  tensor([80,  1,  1, 28])  target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39])  target is  tensor(42)
when input is  tensor([80,  1,  1, 28, 39, 42])  target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39, 42, 39])  target is  tensor(44)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44])  target is  tensor(32)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44, 32])  target is  tensor(49)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49])  target is  tensor(1)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1])  target is  tensor(25)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25])  target is  tensor(38)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38])  target is  tensor(28)
when input is  tensor([80,  

In [6]:
@torch.no_grad()
def estimate_loss():
    out={}
    #testing
    model.eval() #evaluation  mode (dropout is off here)
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() #sampling and challenging the network
    return out

In [7]:
class Head(nn.Module):
    '''one head of self-attention '''
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5

        # Dynamically create tril mask
        tril = torch.tril(torch.ones(T, T, device=x.device))  
        wei = wei.masked_fill(tril == 0, float('-inf'))

        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out


class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    '''Linear Layer followed by a Non-Linearity'''
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    '''Transformer Block: communucation followed by computation'''
    def __init__(self, n_embd, n_head):

        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLLM(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.positional_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) #final layer
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0.0, std=0.02)

    def forward(self, index, targets = None):
        
        B, T = index.shape
        
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.positional_embedding_table(torch.arange(T, device=device))  # Corrects function typo
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
            
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):

        for _ in range(max_new_tokens):

            logits, loss = self.forward(index)

            logits = logits[:,-1,:]

            probs = F.softmax(logits, dim=-1)

            index_next = torch.multinomial(probs, num_samples=1)

            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLLM(vocab_size)
m = model.to(device)

In [8]:
#PyTorch Optimiser
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step: {iter}, train loss:{losses['train']:.3f}, val loss:{losses['val']:.3f}")
    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

Step: 0, train loss:4.401, val loss:4.402
Step: 100, train loss:2.304, val loss:2.371
Step: 200, train loss:1.851, val loss:1.973
Step: 300, train loss:1.600, val loss:1.766
Step: 400, train loss:1.466, val loss:1.673
Step: 500, train loss:1.360, val loss:1.604
Step: 600, train loss:1.287, val loss:1.551
Step: 700, train loss:1.225, val loss:1.533
Step: 800, train loss:1.163, val loss:1.510
Step: 900, train loss:1.111, val loss:1.498
1.1585700511932373


In [9]:
context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500).tolist())
print(generated_chars)

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`